In [3]:
import pandas as pd
import numpy as np

In [4]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

Our economic dataset comprises over 20 sheets, each containing detailed South Africa GDP information spanning from 1993 to 2019. Not all of these sheets are pertinent to our current project. Therefore, our initial task is to identify the relevant sheets, extract them, and subsequently process the data into a usable format.

In the following code block, we extract and preprocess sheets 17 to 26, which contain GDP data categorized by province and economic activity, presented in current prices."

Before running the cell below, ensure you have downloaded - the excel sheet SA_GDP_Data_2020.xlsx

In [5]:
# Specify the name of the Excel file
file_name = 'SA_GDP_Data_2020.xlsx'
main_df = pd.DataFrame()

sheet_numbers = range(17, 26)

# Loop through the list of sheet numbers
for sheet in sheet_numbers:  
    # Create a sheet name by appending 'Table' to the current sheet number
    sheet_name = f'Table {sheet}'

    # Read the Excel file and the desired sheet into a pandas DataFrame
    df = pd.read_excel(file_name, sheet_name=sheet_name)

    # Transpose the DataFrame so the rows become the column
    df_subset = df.transpose()

    # Set the column names of df_subset to be the first row and drop the previous index
    df_subset.columns = df_subset.iloc[0]
    df_subset = df_subset[1:].reset_index(drop=True)

    # Extract the first 18 columns
    df_subset = df_subset.iloc[:, :18]

    # Rename the "industry" column to "year"
    df_subset = df_subset.rename(columns={'Industry': 'year'})
    df_subset["year"] = df_subset["year"].astype(int) # Change the data type from object to int

    # Define features
    features = list(df_subset.columns)[:18]
    delete_columns = [0, 2, 5, 9, 15] # Specify columns to be deleted
    feature_list = [] # Initialize empty list - feature_list

    # Loop through the features and populate feature_list based on conditions
    for x in enumerate(features): # Enumerate attaches the index number to every feature
        if x[0] not in delete_columns:
            feature_list.append(x[1])
            
    province_name = list(df.columns)[0].split("–")[0].strip()
    temp_df = df_subset[feature_list].copy()
    temp_df["province"] = province_name
    
    # Merge the DataFrames vertically
    # main_df = pd.concat([main_df, temp_df])
    # Merge the DataFrames vertically and reset the index
    main_df = pd.concat([main_df, temp_df]).reset_index(drop=True)

# Convert multiple columns to float data type
columns_to_convert = list(main_df.drop(["year", "province"], axis=1).columns)
main_df[columns_to_convert] = main_df[columns_to_convert].astype(float)    

# # Rename the columns (optionally for EDA)
# new_col_names = ["aff", "mq", "manuf", "egw", "constr", "tca", "tsc", "frbs", "ps", "ggs", "tlsp", "gdp"]
# new_columns = dict()
# for old, new in zip(columns_to_convert, new_col_names):
#     new_columns[old] = new
    
# main_df = main_df.rename(columns=new_columns)


In [6]:
main_df.head()

,year,"Agriculture, forestry and fishing",Mining and quarrying,Manufacturing,"Electricity, gas and water",Construction,"Trade, catering and accommodation","Transport, storage and communication","Finance, real estate and business services",Personal services,General government services,Taxes less subsidies on products,GDPR at market prices,province
0,1995,4942.731047,442.546983,17263.258008,1637.212870,3367.844365,10881.091448,6902.876448,16740.441292,3595.257547,8770.439792,6636.710172,81180.409970,Western Cape
1,1996,5297.576761,421.524437,18507.582462,1682.899429,3452.603466,12185.927317,8038.513784,18154.311881,4051.599287,10913.416889,7140.929873,89846.885585,Western Cape
2,1997,5855.780107,230.731253,20147.387588,1715.592387,4057.259535,13213.855870,9116.956886,21358.587992,4578.100207,16852.229046,8023.164478,105149.645349,Western Cape
3,1998,6098.182642,256.872962,20930.731216,1861.780047,4158.673691,14180.325185,10357.759342,22545.582297,5082.493591,12236.036639,9045.114506,106753.552119,Western Cape
4,1999,6187.449779,249.964158,21760.057961,1994.539223,4724.246717,16462.892426,11358.123825,26784.222710,5739.894071,12843.130812,10163.447815,118267.969497,Western Cape


In [7]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 14 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   year                                        225 non-null    int32  
 1   Agriculture, forestry and fishing           225 non-null    float64
 2   Mining and quarrying                        225 non-null    float64
 3   Manufacturing                               225 non-null    float64
 4   Electricity, gas and water                  225 non-null    float64
 5   Construction                                225 non-null    float64
 6   Trade, catering and accommodation           225 non-null    float64
 7   Transport, storage  and communication       225 non-null    float64
 8   Finance, real estate and business services  225 non-null    float64
 9   Personal services                           225 non-null    float64
 10  General govern

### Extract Yearly and Quarterly GDP by Industry

Here we first extract the yearly GDP from 1993 - 2020 from the table 1.

Then we move ahead to extract the GDP for each quarter represented in those years.


In [9]:
# # List all sheet names in the Excel file
# sheet_names = pd.ExcelFile(excel_file_name).sheet_names
# print(sheet_names)

In [10]:
# Defining the excel file name
file = 'SA_GDP_Data_2020.xlsx'

# Specifying the sheet to read
sheet_name = 'Table1 ' 

# Read the specified sheet into a DataFrame
df_gdp = pd.read_excel(file, sheet_name=sheet_name)

#Extract the GDP Values from year 2008 - 2020
GDP_yearly = df_gdp.iloc[15:28]

#Drop the Quar-ter column
GDP_yearly = GDP_yearly.drop('Quar-ter ', axis=1)

#Extract the GDP Values for each quarter from 2008 - 2020
GDP_quarterly = df_gdp.iloc[104:]


In [36]:
#Save the clean/extracted dataframes
main_df.to_csv('SA_gdp_by_province.csv', index=False)
GDP_yearly.to_csv('SA_yearly_gdp.csv', index=False)   
GDP_quarterly.to_csv('SA_quarterly_gdp.csv', index=False)
